In [6]:
import psycopg2
import json

# Function to read the connection configuration from a JSON file
def read_db_config(json_file_path):
    with open(json_file_path, 'r') as file:
        return json.load(file)

configuration = read_db_config('database3.json')

# Function to connect to an existing database
def connect_to_database(json_file_path):
    try:
        # Read the connection configuration from a JSON file
        configuration = read_db_config(json_file_path)

        # Establish the connection to the database
        connection = psycopg2.connect(
            user=configuration['user'],
            password=configuration['password'],
            database=configuration['dbname']  # Make sure this is your database name
        )

        print("Database connection successfully established.")
        return connection

    except Exception as e:
        print(f"Failed to connect to the database: {e}")
        return None

# Connect to the database
connection = connect_to_database('database3.json')

# Function to create the candidates table
def create_candidates_table(connection):
    # Create a new cursor
    cursor = connection.cursor()
    
    # Create the Candidates table
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS Candidates (
            CandidateID SERIAL PRIMARY KEY,
            FirstName VARCHAR(255) NOT NULL,
            LastName VARCHAR(255) NOT NULL,
            Email VARCHAR(255) NOT NULL,
            ApplicationDate DATE NOT NULL,
            Country VARCHAR(255) NOT NULL,
            Experience INT NOT NULL,
            SeniorityLevel VARCHAR(255) NOT NULL,
            TechnologyStack VARCHAR(255) NOT NULL,
            CodeChallengeScore SMALLINT NOT NULL,
            TechnicalInterviewScore SMALLINT NOT NULL,
            Hired BOOLEAN NOT NULL
        );
    """)
    
    # Confirm the creation of the table
    connection.commit()
    
    # Close the cursor
    cursor.close()

    print("Candidates table successfully created.")

# Create the table using the existing connection
create_candidates_table(connection)

# Close the connection
connection.close()


Database connection successfully established.
Candidates table successfully created.


In [7]:
import pandas as pd
csv = "candidates.csv"
df = pd.read_csv(csv, delimiter = ";")

In [8]:
# Function to insert data into the table
def insert_candidate_data(df, connection):
    cursor = connection.cursor()
    insert_query = '''
        INSERT INTO Candidates (FirstName, LastName, Email, ApplicationDate, Country, Experience, SeniorityLevel, TechnologyStack, CodeChallengeScore, TechnicalInterviewScore, Hired)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    for _, row in df.iterrows():
        hired = row['Code Challenge Score'] >= 7 and row['Technical Interview Score'] >= 7
        data_tuple = (row['First Name'], row['Last Name'], row['Email'], row['Application Date'], row['Country'], row['YOE'], row['Seniority'], row['Technology'], row['Code Challenge Score'], row['Technical Interview Score'], hired)
        cursor.execute(insert_query, data_tuple)
    connection.commit()
    print('Data successfully inserted into the Candidates table.')
    cursor.close()

# Connect to the database
connection = connect_to_database('database3.json')  # Connect to the database
insert_candidate_data(df, connection)  # Call the function to insert data passing the DataFrame
connection.close()  # Close the connection


Database connection successfully established.
Data successfully inserted into the Candidates table.
